In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

import xarray as xr

import sys
sys.path.append('../../Climate-Learning/')

import general_purpose.uplotlib as uplt
import general_purpose.cartopy_plots as cplt
import general_purpose.utilities as ut

HOME = '../'

In [ ]:
lon = np.load('../common/lon.npy')
lat = np.load('../common/lat.npy')
LON, LAT = np.meshgrid(lon,lat)

In [ ]:
mfp_kwargs = dict(one_fig_layout=120, figsize=(10,5),
                  projections=[cplt.ccrs.Orthographic(central_latitude=90), cplt.ccrs.PlateCarree()],
                  extents=[None, (-5, 10, 39, 55)],
                  titles=['Geopotential height', 'Soil moisture'],
                 )
cmaps=['RdBu_r', 'BrBG']

## Composite optimal input of CNN

In [ ]:
ds = xr.open_dataset('optimal-input.nc')
ds

In [ ]:
uplt.avg(ds.output.data)

In [ ]:
cplt.mfp(LON, LAT, ds.optimal_input_mean.data, **mfp_kwargs, cmaps=cmaps, fig_num=2)
cplt.mfp(LON, LAT, ds.optimal_input_std.data, **mfp_kwargs, cmaps='Greys', fig_num=3, colorbar='disabled', vmin=0)

### Make it a single figure

In [ ]:
kw = dict(mode='pcolormesh',
        greenwich=True,
        draw_gridlines=False, draw_labels=False,
         )

plt.close(1)
fig = plt.figure(num=1, figsize=(10, 10))


# plot geoplots
projs = np.concatenate([ds.optimal_input_mean.data, ds.optimal_input_std.data], axis=-1)

for i in range(4):
    ax = fig.add_subplot(221 + i, projection=mfp_kwargs['projections'][i%2])

    if i < 2:
        vmin = None
        _mx = np.max(np.abs(projs[...,i]))
        _norm = matplotlib.colors.TwoSlopeNorm(vcenter=0., vmin=-_mx, vmax=_mx)
        cmap = cmaps[i%2]
        title = mfp_kwargs['titles'][i%2] + ' (mean)'
    else:
        _norm = None
        vmin = 0
        cmap = 'Greys'
        title = mfp_kwargs['titles'][i%2] + ' (std)'


    cplt.geo_plotter(ax, LON, LAT, projs[...,i], cmap=cmap, norm=_norm, title=title, vmin=vmin, **kw)

    if i%2:
        ax.set_extent(mfp_kwargs['extents'][1])


fig.tight_layout(w_pad=0)

fig.savefig(f'{HOME}/CNN-OI.pdf')

## Orthogonal Optimal Input

In [ ]:
dso = xr.open_dataset('orth-optimal-input.nc')
dso

In [ ]:
uplt.avg(dso.output.data - dso.ga_output.data)

In [ ]:
kw = dict(mode='pcolormesh',
        greenwich=True,
        draw_gridlines=False, draw_labels=False,
         )

plt.close(1)
fig = plt.figure(num=1, figsize=(10, 10))


# plot geoplots
projs = np.concatenate([dso.optimal_input_mean.data, dso.optimal_input_std.data], axis=-1)

for i in range(4):
    ax = fig.add_subplot(221 + i, projection=mfp_kwargs['projections'][i%2])

    if i < 2:
        vmin = None
        _mx = np.max(np.abs(projs[...,i]))
        _norm = matplotlib.colors.TwoSlopeNorm(vcenter=0., vmin=-_mx, vmax=_mx)
        cmap = cmaps[i%2]
        title = mfp_kwargs['titles'][i%2] + ' (mean)'
    else:
        _norm = None
        vmin = 0
        cmap = 'Greys'
        title = mfp_kwargs['titles'][i%2] + ' (std)'


    cplt.geo_plotter(ax, LON, LAT, projs[...,i], cmap=cmap, norm=_norm, title=title, vmin=vmin, **kw)

    if i%2:
        ax.set_extent(mfp_kwargs['extents'][1])


fig.tight_layout(w_pad=0)

fig.savefig(f'{HOME}/CNN-OI-orth.pdf')

## Make it a single figure with the two

In [ ]:
kw = dict(mode='pcolormesh',
        greenwich=True,
        draw_gridlines=False, draw_labels=False,
         )

plt.close(1)
fig = plt.figure(num=1, figsize=(20, 10))


# plot geoplots
projs = np.concatenate([ds.optimal_input_mean.data, dso.optimal_input_mean.data, ds.optimal_input_std.data, dso.optimal_input_std.data], axis=-1)

for i in range(8):
    ax = fig.add_subplot(241 + i, projection=mfp_kwargs['projections'][i%2])

    if i < 4:
        vmin = None
        _mx = np.max(np.abs(projs[...,i]))
        _norm = matplotlib.colors.TwoSlopeNorm(vcenter=0., vmin=-_mx, vmax=_mx)
        cmap = cmaps[i%2]
        title = mfp_kwargs['titles'][i%2] + ' (mean)'
    else:
        _norm = None
        vmin = 0
        cmap = 'Greys'
        title = mfp_kwargs['titles'][i%2] + ' (std)'


    cplt.geo_plotter(ax, LON, LAT, projs[...,i], cmap=cmap, norm=_norm, title=title, vmin=vmin, **kw)

    if i%2:
        ax.set_extent(mfp_kwargs['extents'][1])


fig.suptitle('Optimal input' + ' '*60 + 'Orthogonal optimal input')

fig.tight_layout(w_pad=0)

fig.savefig(f'{HOME}/CNN-OI-all.pdf')